In [2]:
pip install flask_ngrok

In [3]:
pip install pymorphy2

     |████████████████████████████████| 55 kB 2.2 MB/s 
     |████████████████████████████████| 8.2 MB 7.9 MB/s 


In [ ]:
from flask import Flask, jsonify, request
from flask_ngrok import run_with_ngrok
import pandas as pd
import seaborn as sns
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.stem import WordNetLemmatizer
import re
import pymorphy2
import string
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
from nltk.probability import FreqDist

app = Flask(__name__)
run_with_ngrok(app)


def lemm(data):
  if type(data) == str:
    data = data.split()
  morph = pymorphy2.MorphAnalyzer()
  stop_words = set(stopwords.words('russian'))
  words = [re.sub(r'\d+', '', i.lower()).replace('»', '').replace('«', '').replace('’', '').translate(str.maketrans('', '', string.punctuation)) for i in data if i not in stop_words]
  words = filter(None, words)
  tokens = []
  for i in words:
    tokens.append([morph.parse(w)[0].normal_form for w in nltk.word_tokenize(i)])
  tokens = [sorted(list([j for j in i])) for i in tokens]
  return tokens


def get_tags(data):
  list_tags = []
  for i in data:
    list_list_tag = []
    if 'шар' and 'урна' in i:
      list_list_tag.append('Задача про шары и урну')
    elif 'урна' in i:
      list_list_tag.append('Задача про урну')
    elif 'шар' in i:
      list_list_tag.append('Задача про шары')
    if 'классический' and 'вероятность' in i:
      list_list_tag.append('Классическая вероятность')
    if 'синоним' in i:
      list_list_tag.append('Синонимы')
    if 'значение' and 'слово' in i:
      list_list_tag.append('Значение слова')
    if ('просклонять' or 'склонение') and 'русский язык' in i:
      list_list_tag.append('Склонение')
    if 'цикл' and 'while' in i:
      list_list_tag.append('Цикл while')
    elif 'цикл' and 'for' in i:
      list_list_tag.append('Цикл for')
    elif 'цикл' in i:
      list_list_tag.append('Цикл')
    if 'грамматика' in i:
      list_list_tag.append('Грамматика')
    if 'данные' and 'тип' and 'float' in i:
      list_list_tag.append('Тип данных float')
    elif 'данные' and 'тип' and ('int' or 'integer') in i:
      list_list_tag.append('Тип данных integer')
    elif 'данные' and 'тип' and ('str' or 'string') in i:
      list_list_tag.append('Тип данных string')
    elif 'данные' and 'тип' in i:
      list_list_tag.append('Типы данных')
    if 'pascal' in i:
      list_list_tag.append('Pascal')
    if 'javascript' in i:
      list_list_tag.append('Javascript')
    if 'php' in i:
      list_list_tag.append('Php')
    if 'java' in i:
      list_list_tag.append('Java')
    if 'html' in i:
      list_list_tag.append('Html')
    if 'css' in i:
      list_list_tag.append('Css')
    if 'python' in i:
      list_list_tag.append('Python')
    if 'система' and 'счисление' and 'восьмеричный' in i:
      list_list_tag.append('Восьмеричная система счисления')
    elif 'система' and 'счисление' and 'десятеричный' in i:
      list_list_tag.append('Десятеричная система счисления')
    elif 'система' and 'счисление' and 'двоичный' in i:
      list_list_tag.append('Двоичная система счисления')
    elif 'основание' and 'система' and 'счисление' in i:
      list_list_tag.append('Основание системы счисления')
    elif 'система' and 'счисление' and 'римский' in i:
      list_list_tag.append('Римская система счисления')
    if 'общий' and 'определение' in i:
      list_list_tag.append('Общие определения')
    if 'перевод' in i:
      list_list_tag.append('Перевод')
    if 'дата' in i:
      list_list_tag.append('Дата')
    if 'крещение' and 'русь' in i:
      list_list_tag.append('Крещение Руси')
    if 'война' and 'второй' and 'мировой' in i:
      list_list_tag.append('Вторая мировая война')
    elif 'война' in i:
      list_list_tag.append('Война')
    if 'сражение' in i:
      list_list_tag.append('Сражение')
    if 'византия' in i:
      list_list_tag.append('Византия')
    if 'гриб' and 'царство' in i:
      list_list_tag.append('Царство грибы')
    if 'корень' and 'уравнение' in i:
      list_list_tag.append('Корень уравнения')
    elif 'уравнение' in i:
      list_list_tag.append('Уравнение')
    if 'математика' in i:
      list_list_tag.append('Математика')
    if 'русский язык' in i:
      list_list_tag.append('Русский язык')
    if 'информатика' in i:
      list_list_tag.append('Информатика')
    if 'история' and 'событие' in i:
      list_list_tag.append('Историческое событие')
    if 'история' in i:
      list_list_tag.append('История')
    if 'английский язык' in i:
      list_list_tag.append('Английский язык')
    if 'биология' in i:
      list_list_tag.append('Биология')
    list_tags.append(list_list_tag)
  return list_tags


def get_similar_quests(data, sphere, subsphere, is_solved):
  list_index = []
  for i in df.loc[(df['subject'] == sphere.lower()) & (df['topic'] == subsphere) & (df['is_solved'] == is_solved)].index:
    if len([j for j in df.iloc[i]['tags'].split(', ') if j in data]) / len(data) > 0.5:
      list_index.append(i)
  return df.iloc[list_index]


def create_new_tags_all(sphere, subsphere, title):
  return list(set([''.join(i) for i in get_tags(lemm(title) + lemm(subsphere) + [sphere.lower()]) if i != []]))


def create_new_tags(sphere, subsphere):
  return list(set([''.join(i) for i in get_tags(lemm(subsphere) + [sphere.lower()]) if i != []]))


def get_recommendation(solved_id):
  t = pd.DataFrame(columns = df.columns)
  for i in df.loc[(df['solved_id'] == solved_id)].index:
    t = pd.concat([t, get_similar_quests(df.iloc[i]['tags'].split(', '), df.iloc[i]['subject'], df.iloc[i]['topic'], 0)], ignore_index=True)
    t = t.drop_duplicates()
  t = t.loc[t['author_id'] != solved_id]
  return t


def far_meaning(quest: str, df, sphere, subsphere, threshold=0.5):
  quest = sorted([i[0] for i in lemm(quest.split()) if i != []])
  corpus = [' '.join(i) for i in lemm(df.loc[(df['subject'] == sphere.lower()) & (df['topic'] == subsphere)]['title'])]
  vectorizer = TfidfVectorizer()
  X = vectorizer.fit_transform(corpus)
  y = vectorizer.transform(quest)
  similarities = {}
  for i in range(len(corpus)):
    similarities[i] = sum([i for i in cosine_similarity(X[i], y)[0]])
  similarities = {k: v for k, v in sorted(similarities.items(), key=lambda item: item[1], reverse=True)}
  match = df.iloc[corpus.index(corpus[list(similarities.keys())[0]])]
  if list(similarities.values())[0] < threshold:
    match = ''
  return match


@app.route('/get_topic', methods=['POST'])
def get_answer():
  if request.method == 'POST':
    data = request.get_json()
    
    df = pd.read_csv('uchi.csv')
    if data['subject'] and data['topic'] and data['title'] and request == 'get_questions':
      if far_meaning(data['title'], df, data['subject'], data['topic']) != '':
        val = df.loc[df.index == far_meaning(data['title'], df, data['subject'], data['topic'])['Unnamed: 0']] + get_similar_quests(create_new_tags_all(data['subject'], data['topic'], data['title']), data['subject'], data['topic'], 1)
        val = val.drop_duplicates().index
      else:
        val = get_similar_quests(create_new_tags_all(data['subject'], data['topic'], data['title']), data['subject'], data['topic'], 1).index
    elif sphere and data['topic'] and request == 'get_questions':
      val = get_similar_quests(create_new_tags(data['subject'], data['topic']), data['subject'], data['topic'], 1).index
    elif solved_id and request == 'get_recommendation':
      val = get_recommendation(solved_id).index
    val = [i for i in val.index]
    return jsonify({'question_id': val})
@app.route('/get_tags', methods=['POST'])
def get_answer():
  create_new_tags_all(data['subject'], data['topic'], data['title'])
@app.route('/get_recommendation', methods=['POST'])
def get_answer():

if __name__ == '__main__':
  app.run()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://61a5-35-224-55-187.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


[2021-10-03 05:21:18,801] ERROR in app: Exception on / [GET]
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1953, in full_dispatch_request
    return self.finalize_request(rv)
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1968, in finalize_request
    response = self.make_response(rv)
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 2098, in make_response
    "The view function did not return a valid response. The"
TypeError: The view function did not return a valid response. The function either returned None or ended without a return statement.
127.0.0.1 - - [03/Oct/2021 05:21:18] "GET / HTTP/1.1" 500 -
